# Importing and Exporting Data

Data can be imported into Google BigQuery from a CSV file stored within Google Cloud Storage, or it can be streamed directly into BigQuery from Python code.

Similarly, the results of a query can be exported to Google Cloud Storage as a set of shards, or they can be streamed directly into a file within Cloud Datalab. Note that for larger data sizes, it is recommended to choose the sharded method.

In [1]:
from datalab.context import Context
import datalab.bigquery as bq
import datalab.storage as storage
import pandas as pd
from StringIO import StringIO

# Importing Data

The first step to analyzing and querying your data is importing it. For this demo, we'll create a temporary table in a temporary dataset within BigQuery, using a small data file within Cloud Storage.

## Importing Data from Cloud Storage

In [2]:
%%storage read --object gs://cloud-datalab-samples/cars.csv --variable cars

In [3]:
print cars

Year,Make,Model,Description,Price
1997,Ford,E350,"ac, abs, moon",3000.00
1999,Chevy,"Venture Extended Edition","",4900.00
1999,Chevy,"Venture Extended Edition",Very Large,5000.00
1996,Jeep,Grand Cherokee,"MUST SELL! air, moon roof, loaded",4799.00



In [4]:
# Create the schema, conveniently using a DataFrame example.
df = pd.read_csv(StringIO(cars))
schema = bq.Schema.from_dataframe(df)

# Create the dataset
bq.Dataset('sample').create()

# Create the table
sample_table = bq.Table('sample.cars').create(schema = schema, overwrite = True)

In [ ]:
sample_table.load('gs://cloud-datalab-samples/cars.csv', mode='append',
                  source_format = 'csv', csv_options=bq.CSVOptions(skip_leading_rows = 1))

In [6]:
%%sql
SELECT * FROM sample.cars

Year,Make,Model,Description,Price
1997,Ford,E350,"ac, abs, moon",3000.0
1999,Chevy,Venture Extended Edition,,4900.0
1999,Chevy,Venture Extended Edition,Very Large,5000.0
1996,Jeep,Grand Cherokee,"MUST SELL! air, moon roof, loaded",4799.0


## Importing Data from a DataFrame

In [7]:
cars2 = storage.Item('cloud-datalab-samples', 'cars2.csv').read_from()
df2 = pd.read_csv(StringIO(cars2))
df2

,Year,Make,Model,Description,Price
0,2010,Honda,Civic,NaN,15000.0
1,2015,Tesla,Model S,NaN,64900.0


In [8]:
df2.fillna(value='', inplace=True)
df2

,Year,Make,Model,Description,Price
0,2010,Honda,Civic,,15000.0
1,2015,Tesla,Model S,,64900.0


In [9]:
sample_table.insert_data(df2)
sample_table.to_dataframe()

,Year,Make,Model,Description,Price
0,1997,Ford,E350,"ac, abs, moon",3000.0
1,1999,Chevy,Venture Extended Edition,,4900.0
2,1999,Chevy,Venture Extended Edition,Very Large,5000.0
3,1996,Jeep,Grand Cherokee,"MUST SELL! air, moon roof, loaded",4799.0
4,2010,Honda,Civic,,15000.0
5,2015,Tesla,Model S,,64900.0


# Exporting Data

## Exporting Data to Cloud Storage

In [ ]:
project = Context.default().project_id
sample_bucket_name = project + '-datalab-samples'
sample_bucket_path = 'gs://' + sample_bucket_name
sample_bucket_object = sample_bucket_path + '/tmp/cars.csv'
print 'Bucket: ' + sample_bucket_name
print 'Object: ' + sample_bucket_object

In [11]:
sample_bucket = storage.Bucket(sample_bucket_name)
sample_bucket.create()
sample_bucket.exists()

True

In [ ]:
table = bq.Table('sample.cars')
table.extract(destination = sample_bucket_object)

In [13]:
%%storage list --bucket $sample_bucket_path

Name,Type,Size,Updated
tmp/cars.csv,application/octet-stream,230,2016-05-09 19:32:37.854000+00:00


In [14]:
%%storage read --object $sample_bucket_object --variable data

In [15]:
print data

Year,Make,Model,Description,Price
1997,Ford,E350,"ac, abs, moon",3000
1999,Chevy,Venture Extended Edition,,4900
1999,Chevy,Venture Extended Edition,Very Large,5000
1996,Jeep,Grand Cherokee,"MUST SELL! air, moon roof, loaded",4799



## Exporting Data to a Local File

In [16]:
table.to_file('/tmp/cars.csv')

In [17]:
%%bash
ls -l /tmp/cars.csv

-rw-r--r-- 1 root root 299 May  9 19:33 /tmp/cars.csv


In [18]:
lines = None
with open('/tmp/cars.csv') as datafile:
  lines = datafile.readlines()
print ''.join(lines)

Year,Make,Model,Description,Price
1997,Ford,E350,"ac, abs, moon",3000.0
1999,Chevy,Venture Extended Edition,,4900.0
1999,Chevy,Venture Extended Edition,Very Large,5000.0
1996,Jeep,Grand Cherokee,"MUST SELL! air, moon roof, loaded",4799.0
2010,Honda,Civic,,15000.0
2015,Tesla,Model S,,64900.0



# Cleanup

In [19]:
sample_bucket.item('tmp/cars.csv').delete()
sample_bucket.delete()
bq.Dataset('sample').delete(delete_contents = True)